In [1]:
import pygrank
import networkx
import fairbench as fb

The default pygrank backend has been set to "numpy" by the file /home/maniospas/.pygrank/config.json
Set your preferred backend as one of ["numpy", "pytorch", "tensorflow", "dask", "torch_sparse", "matvec", "sparse_dot_mkl"] and "reminder": false in that file to remove this message from future runs.


## Load data

In [2]:
_, graph, communities = next(pygrank.load_datasets_multiple_communities(["citeseer"], graph_api=networkx))
print(graph, " "*20)  # clear output

REQUIRED CITATION: Please visit the url https://linqs.soe.ucsc.edu/data for instructions on how to cite the dataset citeseer in your research


Graph with 3327 nodes and 4676 edges                     


In [3]:
seeds = pygrank.to_signal(graph, communities[0]) 
sensitive = pygrank.to_signal(graph, communities[1]) 
train, test = pygrank.split(seeds, 0.5)
dict(test)  # what graph signals look like

{'100157': 1.0,
 '364207': 1.0,
 '38848': 0.0,
 'bradshaw97introduction': 0.0,
 'bylund99coordinating': 0.0,
 'dix01metaagent': 0.0,
 'gray99finding': 0.0,
 'labrou01standardizing': 0.0,
 'labrou99agent': 0.0,
 'nodine98overview': 0.0,
 'nodine99active': 0.0,
 'wagner97artificial': 1.0,
 '100598': 0.0,
 '455651': 0.0,
 'marquez00machine': 0.0,
 'punyakanok01use': 0.0,
 '101570': 0.0,
 'krasnogor00memetic': 0.0,
 '10227': 0.0,
 '131669': 0.0,
 'sima00computational': 0.0,
 'sima01computational': 0.0,
 '103027': 1.0,
 '140169': 0.0,
 '105684': 0.0,
 'weiss00building': 0.0,
 '106003': 0.0,
 '104129': 0.0,
 'singhal99document': 0.0,
 '106339': 0.0,
 'amer-yahia00boundingschemas': 0.0,
 'arlein99making': 0.0,
 'cluet99using': 0.0,
 'shasha02algorithmics': 0.0,
 '108321': 0.0,
 '184462': 0.0,
 '73962': 0.0,
 '83140': 0.0,
 '108573': 0.0,
 '448486': 0.0,
 'bailey01analysis': 0.0,
 'bailey02eventconditionaction': 0.0,
 'tova99active': 0.0,
 '108580': 0.0,
 'cohen01learning': 0.0,
 'raman01potte

In [4]:
test.np  # internal array representation

array([1., 1., 0., ..., 0., 0., 0.], shape=(3327,))

In [5]:
test.filter(exclude=train)  # the array representation without elements for which train==1

array([1., 1., 0., ..., 0., 0., 0.], shape=(3029,))

## Run a node ranking algorithm

In [6]:
algorithm = pygrank.PageRank(alpha=0.85)
scores = algorithm(train)
scores = scores.np
scores

array([0.50342929, 0.27577646, 0.27670741, ..., 0.00812659, 0.00146229,
       0.00566413], shape=(3327,))

In [7]:
import numpy as np
sensitive = pygrank.to_signal(graph, communities[1])
sensitive=fb.Dimensions(fb.categories@np.array(sensitive.np,dtype=int))
sensitive

0,[1 1 1 ... 1 0 1]
1,[0 0 0 ... 0 1 0]


In [10]:
report = fb.reports.pairwise(labels=test.np, scores=scores, sensitive=sensitive)
report.show(env=fb.export.ConsoleTable)

                                                                                                  
                                    multidim                                                      
                                      avgscore          auc        tophr       toprec        topf1
min                                      0.013        0.780            0            0            0
max                                      0.109        0.780        0.333        0.003        0.007
wmean                                    0.090        0.780        0.266        0.003        0.005
mean                                     0.061        0.780        0.167        0.002        0.003
maxbarea                                 0.172            0                                       
maxrel                                   0.881            0            1            1            1
maxdiff                                  0.096            0        0.333        0.003        0.007
gini      

In [12]:
report.min.tophr.show(depth=2)


##### min tophr #####
|This reduction of a measure is the minimum of the hit ratio of top 
|recommendations.
|Value: 0.000 min tophr

 ***** 0 *****
 |This is the value for group '0'.
 |Value: 0.333 tophr
 
   (0.0, 2659.0)
   ▎       █
   ▎       █
   ▎       █
   ▎       █
   ▎       █
   ▎▬*▬▬-▬▬+
   (3.0, 0.0)
   
    * top                               3 
    - tp                                1 
    + samples                           2659 
 
 ***** 1 *****
 |This is the value for group '1'.
 |Value: 0.000 tophr
 
   (0.0, 668.0)
   ▎       █
   ▎       █
   ▎       █
   ▎       █
   ▎       █
   ▎▬*▬▬-▬▬+
   (3.0, 0.0)
   
    * top                               3 
    - tp                                0 
    + samples                           668 
 

 


In [13]:
report.min.auc.explain.show()


##### min auc explain #####
|This explanation of a reduction of a measure is the minimum of the area under 
|curve of the receiver operating characteristics viewed for inner details.
|Computations cover several cases.

 ***** samples *****
 |This is the sample count.
 |Computations cover several cases.
 
   (0.0, 2659.0)
   ▎ █
   ▎ █
   ▎ █
   ▎ █
   ▎ █
   ▎▬*
   (1.0, 0.0)
   
    * 0                                 2659 samples
 
 ***** roc *****
 |This is the receiver operating characteristics curve.
 |Computations cover several cases.
 
   (0.0, 1.0)
   ▎                                   ****
   ▎                       *************   
   ▎      ******************               
   ▎     **                                
   ▎     *                                 
   ▎    **                                 
   ▎   **                                  
   ▎  **                                   
   ▎ **                                    
   ***▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬

In [14]:
sensitive = pygrank.to_signal(graph, communities[1])
fair_algorithm = pygrank.LFPR(alpha=0.85, redistributor="original")

fair_scores = fair_algorithm(train, sensitive=sensitive).np
sensitive=fb.Dimensions(fb.categories@np.array(sensitive.np,dtype=int))
fair_report = fb.reports.pairwise(labels=test.np, scores=fair_scores, sensitive=sensitive)
fair_report.show(env=fb.export.ConsoleTable)

                                                                                                  
                                    multidim                                                      
                                      avgscore          auc        tophr       toprec        topf1
min                                      0.076        0.787            0            0            0
max                                      0.076        0.787        0.667        0.007        0.013
wmean                                    0.076        0.787        0.533        0.005        0.011
mean                                     0.076        0.787        0.333        0.003        0.007
maxbarea                                 0.396            0                                       
maxrel                                   0.000            0            1            1            1
maxdiff                                  0.000            0        0.667        0.007        0.013
gini      

In [15]:
fair_report.auc.min.explain.show()


##### min explain #####
|This explanation of a reduction is the minimum viewed for inner details.
|Computations cover several cases.

 ***** samples *****
 |This is the sample count.
 |Computations cover several cases.
 
   (0.0, 2659.0)
   ▎ █
   ▎ █
   ▎ █
   ▎ █
   ▎ █
   ▎▬*
   (1.0, 0.0)
   
    * 0                                 2659 samples
 
 ***** roc *****
 |This is the receiver operating characteristics curve.
 |Computations cover several cases.
 
   (0.0, 1.0)
   ▎                                     **
   ▎                       *************** 
   ▎       *****************               
   ▎      **                               
   ▎     **                                
   ▎   ***                                 
   ▎  **                                   
   ▎ **                                    
   ▎**                                     
   **▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬
                                 (1.0, 0.0)
   
    * 0 


 


In [16]:
compare = fb.Progress("Compare avgscore")
compare.instance("PPR", report.maxrel.avgscore)
compare.instance("LFPRO", fair_report.maxrel.avgscore)
compare = compare.build()
compare.show()


##### Compare avgscore #####
|This is tracking progress across Compare avgscore.
|Computations cover several cases.

  (0.0, 0.8809895029359326)
  ▎ █   
  ▎ █   
  ▎ █   
  ▎ █   
  ▎ █   
  ▎▬*▬▬-
  (2.0, 0.0)
  
   * PPR maxrel avgscore                 0.881 
   - LFPRO maxrel avgscore               0.000 





In [17]:
compare.explain.show(depth=1)


##### Compare avgscore explain #####
|This explanation is tracking progress across Compare avgscore viewed for 
|inner details.
|Computations cover several cases.

 ***** 0 *****
 |This is the value for group '0'.
 |Computations cover several cases.
 
   (0.0, 0.1088187356248636)
   ▎ █   
   ▎ █   
   ▎ █  █
   ▎ █  █
   ▎ █  █
   ▎▬*▬▬-
   (2.0, 0.0)
   
    * PPR maxrel avgscore               0.109 avgscore
    - LFPRO maxrel avgscore             0.076 avgscore
 
 ***** 1 *****
 |This is the value for group '1'.
 |Computations cover several cases.
 
   (0.0, 0.0761346558461076)
   ▎    █
   ▎    █
   ▎    █
   ▎    █
   ▎    █
   ▎▬*▬▬-
   (2.0, 0.0)
   
    * PPR maxrel avgscore               0.013 avgscore
    - LFPRO maxrel avgscore             0.076 avgscore
 

 


## How about some post-processing and avgscore comparisons? (tricky result interpretation)

In [25]:
sensitive = pygrank.to_signal(graph, communities[1]).np

simple_scores = fair_scores
simple_scores = simple_scores*sensitive/np.sum(simple_scores*sensitive) + simple_scores*(1-sensitive)/np.sum(simple_scores*(1-sensitive))

/tmp/ipykernel_27834/3534655343.py:4: RuntimeWarning: invalid value encountered in divide
  simple_scores = simple_scores*sensitive/np.sum(simple_scores*sensitive) + simple_scores*(1-sensitive)/np.sum(simple_scores*(1-sensitive))


In [24]:
sensitive = pygrank.to_signal(graph, communities[1])
sensitive=fb.Dimensions(fb.categories@np.array(sensitive.np,dtype=int))

simple_report = fb.reports.pairwise(labels=test.np, scores=simple_scores, sensitive=sensitive)
simple_report.show(env=fb.export.ConsoleTable(sideways=False), depth=10)


multidim min min avgscore
                                          0            1
positives                             1.000            1
samples                                2659          668

multidim min min auc
                                          0
samples                                2659

multidim min min tophr
                                          0            1
top                                       3            3
tp                                        2            0
samples                                2659          668

multidim min min toprec
                                          0            1
top                                       3            3
tp                                        2            0
ap                                      298            0
samples                                2659          668

multidim min min topf1
                                          0            1
top                                       3      

In [22]:
compare = fb.Progress("Compare avgscore")
compare.instance("PPR", report.maxrel.avgscore)
compare.instance("LFPRO", fair_report.maxrel.avgscore)
compare.instance("Simple", simple_report.maxrel.avgscore)
compare = compare.build()
compare.show()


##### Compare avgscore #####
|This is tracking progress across Compare avgscore.
|Computations cover several cases.

  (0.0, 0.8809895029359326)
  ▎ █      
  ▎ █     █
  ▎ █     █
  ▎ █     █
  ▎ █     █
  ▎▬*▬▬-▬▬+
  (3.0, 0.0)
  
   * PPR maxrel avgscore                 0.881 
   - LFPRO maxrel avgscore               0.000 
   + Simple maxrel avgscore              0.749 



